# Wrangle Exercises

## Data Acquisition

These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.

This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

### 1. Read the case, department, and source data into their own spark dataframes.


In [10]:
# Read in CSV files 

case = (spark.read.csv("case.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )
department = (spark.read.csv("dept.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )
source = (spark.read.csv("source.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )

### 2. Let's see how writing to the local disk works in spark:

- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
- Inspect your folder structure. What do you notice?

In [11]:
case.write.csv("case_csv", mode="overwrite")
case.write.json("case_json", mode="overwrite")

department.write.csv("department_csv", mode="overwrite")
department.write.json("department_json", mode="overwrite")

source.write.csv("source_csv", mode="overwrite")
source.write.json("source_json", mode="overwrite")

### 3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [13]:
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [14]:
department.dtypes

[('dept_division', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string')]

-------------------------------------------------------------------
### 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

### 2. How many Stray Animal cases are there?

### 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

### 4. Convert the council_district column to a string column.


### 5. Extract the year from the case_closed_date column.


### 6. Convert num_days_late from days to hours in new columns num_hours_late.


### 7. Join the case data with the source and department data.


### 8. Are there any cases that do not have a request source?


### 9. What are the top 10 service request types in terms of number of requests?


### 10. What are the top 10 service request types in terms of average days late?


### 11. Does number of days late depend on department?


### 12. How do number of days late depend on department and request type?